In [1]:
def chi(term, cl, X, Y):
    """ chisquare（卡方）检验算法，最常用特征选择算法之一 """
    N = len(Y)
    A = B = C = D = 0
    for i, x in enumerate(X):
        flag = False
        #print("i:%s,x:%s" % (i,x))
        #for k in x.keys():
        for k in x.split(' '):
            #print("k:%s,term:%s" %(k, term))
            if k == term:
                flag = True
                break
        
        if flag == True:
            if str(Y[i]) == str(cl):
                A = A + 1
            else:
                B = B + 1
        else:
            if str(Y[i]) == str(cl):
                C = C + 1
            else:
                D = D + 1
    print("A:%d, B:%d, C:%d, D:%d" % (A,B,C,D))
    re = int((N * (A * D - B * C) * (A * D - B * C)) / ((A + B) * (A + C) * (B + D) * (C + D)))
    return re

                
def dump_feature(labels, dic, X, Y, filePath):
    """ 遍历词袋中的词，然后与训练集数据执行卡方检验算法，并保存结果 """
    fp = open(filePath, 'w')
    dic_num = len(dic)
 
    # lable是类别列表[0, 1]
    # 保存在txt文件中，标签为0的一行 标签为1的一行
    for cl in labels:
        fp.write(str(cl))
        i = 0
        for term in dic.keys():
            #print("term:%s, c1:%s" % (term, cl))
            if ((i+1)%1000 == 0):
                print("term %d of %d" % (i+1, dic_num))
            s = chi(term, cl, X, Y)
            #print('当前检测词为: {} {}'.format(term, s))
            fp.write(' ' + term + ':' + str(s))
            i+=1
            break;
        fp.write('\n')
        fp.flush()
    fp.close()

In [2]:
import jieba
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#impot utils

TRAIN_FILE   = './data/train_data.csv'
TEST_FILE    = './data/test_data.csv'
SUBMIT_FILE  = './data/tfidf_lr_chi_submission.csv'
FEATURE_FILE = './data/feature_word.txt'

class TF_IDF_LR():
    def __init__(self, input_file,  input_file_encoding, stop_words_file=False):
        '''
        input_file:          输入文件名
        input_file_encoding：输入文件编码格式
        stop_words_file：    停顿词文件名，无停顿词时设置为False
        '''
        self.input_file = input_file
        self.input_file_encoding = input_file_encoding
        self.stop_words_file = stop_words_file
        
    def load_data(self):
        '''
        利用pandas加载数据到dataframe
        '''
        return pd.read_csv(self.input_file, encoding=self.input_file_encoding)
    
    def get_data(self, test_size=0.1):
        '''
        取得数据的特征列和标签列，对特征列进行分词的处理，然后划分数据集为训练集和测试集
        '''
        df = self.load_data()
        self.X = df['comment'].apply(self.jieba_tokenizer)
        self.y = df['label']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=test_size)

    def dump_features(self, file_path, num_feautures):
        vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=num_feautures)
 
        features = vectorizer.fit_transform(self.X)

        #print("由此可知语料库汇总词的总数量就是词袋的向量长度，且每一个词汇对应着它出现的顺序和频率")
        #print('feature value: ', features.toarray())
        #print('feature name: ', features.get_feature_names())
        #print('vectorizer.vocabulary_: ', vectorizer.vocabulary_)
        #print('feature count: ', len(vectorizer.vocabulary_))
        
        labels = [0, 1]

        print('开始进行卡方检测...')
        dump_feature(labels, vectorizer.vocabulary_, self.X, self.y, filePath=file_path)
        
    def train(self):
        #出现频率最高的几个，对评价无影响的词,最后未使用
        #excludeWords = {'的','了','我','看','电影','在'}
        excludeWords = {}
        
        #没有max_features:0.812344
        #max_features = 500    #0.766281
        #max_features = 1000   #0.772181
        #max_features = 1500   #0.794872
        #max_features = 2000   #0.795326
        #max_features = 3000   #0.804402
        #max_features = 4000   #0.804629
        #max_features = 5000   #0.809621
        #max_features = 6000   #0.811436
        #max_features = 7000   #0.810302
        #max_features = 8000   #0.811436
        #max_features = 9000   #0.815975
        max_features = 10000  #0.811436
        #max_features = 11000   #0.818017
        #max_features = 12000   #0.815067
        #max_features = 15000    #0.812798
        #max_features = 11000 
        self.tfidf_vec = TfidfVectorizer(smooth_idf=1,
                            analyzer='word',
                            encoding='utf-8',
                            preprocessor=None,
                            ngram_range=(1, 2),
                            max_features = max_features,
                            stop_words=excludeWords)   #812798
                                   
    
        # 使用tfidf的方式，将原始训练和测试文本转化为特征向量。
        X_tfidf_train = self.tfidf_vec.fit_transform(self.X_train)
        X_tfidf_test = self.tfidf_vec.transform(self.X_test)
    
        lr = LogisticRegression()
        lr = lr.fit(X_tfidf_train, self.y_train)

        print('Test set accuracy: %3f' % lr.score(X_tfidf_test, self.y_test))
        
        self.model = lr
    def jieba_tokenizer(self,x):
        '''
        对输入进行分词处理，然后词语之间以空格分割的形式返回字符串
        '''
        words =  jieba.cut(x,cut_all=True)
        return " ".join(words)
    
    def predict(self, test_file, submit_file):
        test_df = pd.read_csv(test_file,  encoding='utf-8', header=0)
        #将行按空格分割
        X = test_df['comment'].apply(self.jieba_tokenizer) 
        
        #将词转换为向量
        X_vect = self.tfidf_vec.transform(X)
        
        #预测
        y_pred = self.model.predict(X_vect)
        
        #预测结果保存
        res = pd.DataFrame({'label':y_pred})
        res.to_csv(submit_file, index=False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
if __name__ == '__main__':
    tf_idf = TF_IDF_LR(TRAIN_FILE, 'utf-8')
    tf_idf.get_data()
    tf_idf.dump_features(FEATURE_FILE, 15000)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.749 seconds.
Prefix dict has been built succesfully.


开始进行卡方检测...
A:17, B:33, C:19091, D:24927
A:33, B:17, C:24927, D:19091


In [9]:
tf_idf = TF_IDF_LR(TRAIN_FILE, 'utf-8')
tf_idf.get_data()
vectorizer = CountVectorizer(analyzer="word",
                     tokenizer=None,
                     preprocessor=None,
                     stop_words=None,
                     max_features=1000)

features = vectorizer.fit_transform(tf_idf.X)

print("由此可知语料库汇总词的总数量就是词袋的向量长度，且每一个词汇对应着它出现的顺序和频率")
print('feature value: ', features.toarray())
#print('feature name: ', features.get_feature_names())
features.f

由此可知语料库汇总词的总数量就是词袋的向量长度，且每一个词汇对应着它出现的顺序和频率
feature value:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
A=17
B=33
C=19091
D=24927
print((1 * (A * D - B * C) * (A * D - B * C)) / ((A + B) * (A + C) * (B + D) * (C + D)))

A=33
B=17
C=24927
D=19091
print((1 * (A * D - B * C) * (A * D - B * C)) / ((A + B) * (A + C) * (B + D) * (C + D)))


4.0523087001575904e-05
4.0523087001575904e-05


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv('./data/train_data.csv', encoding='utf-8')